In [43]:
import datetime
import json
import numpy as np
import pandas as pd
import pickle
import sqlite3
import warnings



from pandas.core.common import SettingWithCopyWarning
from pandas.tseries.offsets import MonthEnd
from scipy.stats import linregress

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [31]:
with open("cfg.json", "r") as jin:
    cfg = json.load(jin)

# rewrite to ensure formatting
with open("cfg.json", "w") as jout:
    json.dump(cfg, jout, indent=4)

In [32]:
conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["install_db"])
cursor_proj = conn_proj.cursor()

query_g = """
SELECT *

FROM grouped_df

"""


g_df = pd.read_sql_query(query_g, conn_proj)

In [34]:
def get_slope(array):
    y = np.array(array)
    x = np.arange(len(y))
    slope, intercept, r_value, p_value, std_err = linregress(x,y)
    return slope

In [35]:
g_df = g_df.sort_values(by = 'month')

g_df['rolling_slope'] = g_df.groupby(
    'zipcode')['count_per_home'].rolling(window=6,min_periods=6).apply(
    get_slope, raw=False).reset_index(0, drop=True)

The following cells update the electricity price with monthly values

In [20]:
elec = pd.read_csv(cfg["data_dir"] + cfg["monthly_e_price"])

elec['description'] = elec.description.apply(lambda x: x.split(': ')[-1])

In [21]:
query_zip_state = """
SELECT *

FROM zipcode

LEFT JOIN state on zipcode.state_id == state.id

"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

zip_state_df = pd.read_sql_query(query_zip_state, conn_proj)
    
conn_proj.close

<function Connection.close>

In [22]:
zip_state_df['zipcode'] = zip_state_df['zipcode'].apply(lambda a: str(a).zfill(5))

In [23]:
zip_rates = zip_state_df.merge(elec, left_on = 'name', right_on = 'description', how = 'left')

In [24]:
min_mo = datetime.strptime(cfg["min_date"],"%d-%b-%Y (%H:%M:%S.%f)") + MonthEnd(1)

In [44]:
ddict = {}

for col in zip_rates.columns:
    try:
        col_name = datetime.datetime.strptime(col, '%b %Y') + MonthEnd(1)
        month = int(round((col_name - min_mo)/np.timedelta64(1, 'M'),0))
        ddict[month] = dict(zip(zip_rates.zipcode,zip_rates[col]))
    except:
        pass

In [46]:
def price_map(x):
    try:
        m = x[0]
        z = x[1]
        
        return ddict[m][z]
    except:
        return np.nan

In [47]:
g_df['cost_electricity'] = [price_map(x) for x in zip(g_df.month,g_df.zipcode)]
g_df = g_df.dropna()

In [49]:
conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["install_db"])
cursor_proj = conn_proj.cursor()

g_df.to_sql("g_df2", conn_proj, if_exists="replace", index=False)

conn_proj.commit

<function Connection.commit>